# 0.0.1-iw-lib-experimentation

In [ ]:
from proxy import Proxy
from urllib import request #import urlopen

proxy = Proxy()
# getting the current proxy
cur_proxy = proxy.proxy
handler = request.ProxyHandler(cur_proxy)
opener = request.build_opener(handler)
request.install_opener(opener)

response = request.urlopen("https://financialmodelingprep.com/api/company/profile/AAPL,GOOG?datatype=json")
data = response.read().decode("utf-8")

print(json.loads(data))

In [8]:
import requests
url = ("https://financialmodelingprep.com/api/v3/financials/income-statement/AAPL")
response = requests.get(url, params={})

In [9]:
response.json()

{'symbol': 'AAPL',
 'financials': [{'date': '2018-09-29',
   'Revenue': '265595000000.0',
   'Revenue Growth': '0.1586',
   'Cost of Revenue': '163756000000.0',
   'Gross Profit': '101839000000.0',
   'R&D Expenses': '14236000000.0',
   'SG&A Expense': '16705000000.0',
   'Operating Expenses': '30941000000.0',
   'Operating Income': '70898000000.0',
   'Interest Expense': '0.0',
   'Earnings before Tax': '72903000000.0',
   'Income Tax Expense': '13372000000.0',
   'Net Income - Non-Controlling int': '0.0',
   'Net Income - Discontinued ops': '0.0',
   'Net Income': '59531000000.0',
   'Preferred Dividends': '0.0',
   'Net Income Com': '59531000000.0',
   'EPS': '12.01',
   'EPS Diluted': '11.91',
   'Weighted Average Shs Out': '4829926000.0',
   'Weighted Average Shs Out (Dil)': '4955377000.0',
   'Dividend per Share': '2.72',
   'Gross Margin': '0.3834',
   'EBITDA Margin': '0.316',
   'EBIT Margin': '0.2745',
   'Profit Margin': '0.224',
   'Free Cash Flow margin': '0.2414',
   'EBI

In [31]:
from urllib import request

import json
def get_data(url, **kwargs):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = request.urlopen(url)
    data = response.read().decode("utf-8")
    
    if 'datatype' in kwargs:
        if 'json' == kwargs['datatype']:
            data = json.loads(data)
        
    return json.loads(data)

url = ("https://financialmodelingprep.com/api/v3/financials/income-statement/AAPL,FB,GOOG")
print(get_data(url))

{'financialStatementList': [{'symbol': 'GOOG', 'financials': [{'date': '2018-12', 'Revenue': '136819000000.0', 'Revenue Growth': '', 'Cost of Revenue': '59549000000.0', 'Gross Profit': '77270000000.0', 'R&D Expenses': '21419000000.0', 'SG&A Expense': '24459000000.0', 'Operating Expenses': '45878000000.0', 'Operating Income': '31392000000.0', 'Interest Expense': '114000000.0', 'Earnings before Tax': '34913000000.0', 'Income Tax Expense': '4177000000.0', 'Net Income - Non-Controlling int': '', 'Net Income - Discontinued ops': '', 'Net Income': '30736000000.0', 'Preferred Dividends': '', 'Net Income Com': '30736000000.0', 'EPS': '44.22', 'EPS Diluted': '43.70', 'Weighted Average Shs Out': '695000000.0', 'Weighted Average Shs Out (Dil)': '750000000.0', 'Dividend per Share': '', 'Gross Margin': '0.56476074229456', 'EBITDA Margin': '0.32204591467559', 'EBIT Margin': '0.25600976472566', 'Profit Margin': '0.22464716157844', 'Free Cash Flow margin': '', 'EBITDA': '44062000000.0', 'EBIT': '35027

## Endpoints for `financialmodellingprep`

In [119]:
BASE_URL = 'https://financialmodelingprep.com/api/v3'

import pandas as pd
from urllib.parse import urlencode
from urllib.parse import urljoin
import inspect
import requests

class FMP(object):
    """ A FinancialModellingPrep API wrapper. """
    
    def __init__(self):
        self.requester = self.Requester()
        self._attach_endpoints()
        
    def _attach_endpoints(self):
        """ Generate and attach endpoints """
        for name, endpoint in inspect.getmembers(self):
            if (inspect.isclass(endpoint) 
                and issubclass(endpoint, self._Endpoint) 
                and endpoint is not self._Endpoint):
                endpoint_instance = endpoint(self.requester)
                setattr(self, endpoint.name, endpoint_instance)
    
    class Requester(object):
        """ An object for making API requests """
        
        def GET(self, url, params={}):
            """ Make a GET request 
            
            :param url: String. FMP endpoint.
            : param params: Dictionary. Query params to include in GET.
            """           
            params.setdefault('datatype', 'json')
            print (f'GET: {url} with params: {params}')
            response = requests.get(url, params=params)
            if 200 != response.status_code:
                error = 'HTTPError: {}'.format(response.status_code)
                return {'success': False, 'error': error}
            try:
                return response.json()
            except ValueError as err:
                return {'success': False, 'error': err}
        
    class _Endpoint(object):
        """ Base class of an endpoint """
        url = 'https://financialmodelingprep.com/api/v3'
        
        def __init__(self, requester):
            self.requester = requester            
        
        def _build_path(self, path, tickers):
            return path + '/' + ','.join(map(str, tickers))
        
        def _GET(self, path, tickers, params={}):
            """ Make a GET request to this endpoint
            
            :param path: String. Path to append to the endpoint url.
            :param params: Dictionary. These are query params.
            """
            path = self._build_path(path, tickers)
            url = urljoin(self.url, path)
            return self.requester.GET(url, params)
    
    class CompanyValuation(_Endpoint):
        """ FMP Company Profile API endpoints. """
        name = 'company_valuation'
        
        def profile(self, tickers, params={}):
            """ 
            Companies profile (Price, Beta, Volume Average, Market Capitalisation, 
            Last Dividend, 52 week range, stock price change, stock price change 
            in percentage, Company Name, Exchange, Description, Industry, Sector, 
            CEO, Website and image). 
            
            Hourly, JSON
            """
            params['datatype'] = 'json'
            return self._GET('company/profile', tickers, params)
        
        def income_statement(self, tickers, period=None, datatype='json', params={}):
            """
            Get the tickers' income statments 
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            if datatype in ['json', 'csv']:
                params['datatype'] = datatype
            return self._GET('financials/income-statement', tickers, params)
        
        def balance_sheet_statement(self, tickers, period=None, datatype='json', params={}):
            """
            Get the tickers' balance sheets
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            if datatype in ['json', 'csv']:
                params['datatype'] = datatype
            return self._GET('financials/balance-sheet-statement', tickers, params)
        
        def cash_flow_statement(self, tickers, period=None, datatype='json', params={}):
            """
            Get the tickers' cash flow statements
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            if datatype in ['json', 'csv']:
                params['datatype'] = datatype
            return self._GET('financials/cash-flow-statement', tickers, params)
        
        def financial_ratios(self, tickers, params={}):
            """
            Get the tickers' cash flow statements
            :param tickers: List. The tickers to query.
            """
            return self._GET('financial-ratios', tickers, params)
        
        def enterprise_value(self, tickers, period=None, params={}):
            """
            Get the tickers' company enterprise value
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            return self._GET('enterprise-value', tickers, params)
        
        def key_metrics(self, tickers, period=None, params={}):
            """
            Get the tickers' company key metrics
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            return self._GET('company-key-metrics', tickers, params)

        def financial_growth(self, tickers, period=None, params={}):
            """
            Get the tickers' financial growth metrics
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            return self._GET('financial-growth', tickers, params)
        
        def company_rating(self, tickers, params={}):
            """
            Get the tickers' ratings. Calculated daily.
            :param tickers: List. The tickers to query.
            """
            return self._GET('company/rating', tickers, params)
        
        def discounted_cash_flow_value(self, tickers, params={}):
            """
            Get the tickers' discounted cash flow.
            Calculated in realtime.
            :param tickers: List. The tickers to query.
            """
            return self._GET('company/discounted-cash-flow', tickers, params)
        
        def historical_discounted_cash_flow_value(self, tickers, period=None, params={}):
            """
            Get the tickers' financial growth metrics
            :param tickers: List. The tickers to query.
            :param period: String. The period of the income statement.
                Options: 'quarter'
            """
            if 'quarter' == period:
                params['period'] = 'quarter'
            return self._GET('company/historical-discounted-cash-flow', tickers, params)
        

In [120]:
tickers=['AAPL']
client = FMP()
client.company_valuation.historical_discounted_cash_flow_value(tickers)

GET: https://financialmodelingprep.com/api/company/historical-discounted-cash-flow/AAPL with params: {'datatype': 'json'}


{'AAPL': {}}